In [1]:
!pip install -q tensorflow-recommenders
!pip install -q scann
!pip install -q azure-storage-blob
!pip install -q tensorboard
!pip install -q tensorflow-datasets

In [2]:
import os
import pprint
import tempfile
import pandas as pd

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2023-03-04 15:31:49.467917: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 15:31:50.872084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-04 15:31:50.872161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-04 15:31:50.872167: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [4]:
import tensorflow_recommenders as tfrs

In [15]:

# run main function
path = "/mnt/c/mldata/aliccp"
log_path = "/mnt/c/mldata/aliccp"

df_train = pd.read_parquet(path+'/train_processed/')
print(len(df_train))

#filter out long tail products and reduce size of dataset
df_train['counter'] = 1
df_agg = df_train[['item_id','counter']].groupby('item_id').count()
df_agg = df_agg.sort_values("counter",ascending=False)[:10000]
df_train = pd.merge(df_agg, df_train, on=['item_id', 'item_id'], how='left')
print(len(df_train))
df_train['counter'] = 1
cust_agg = df_train[['user_id','counter']].groupby('user_id').count()
cust_agg = cust_agg.sort_values("counter",ascending=False)[:8000]
df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')
print(len(df_train))

41999771
9477639


/tmp/ipykernel_12568/425168829.py:17: FutureWarning: Passing 'suffixes' which cause duplicate columns {'counter_y'} in the result is deprecated and will raise a MergeError in a future version.
  df_train = pd.merge(cust_agg, df_train, on=['user_id', 'user_id'], how='left')


1473399


In [16]:
train = df_train[df_train['click']==1][['item_id','user_id']]
features = ['item_id','user_id']
for i in features:
    train[i] = pd.to_numeric(train[i], errors='coerce',downcast="integer")
train= train.dropna()

items = train['item_id'].unique()
items = tf.convert_to_tensor(items, dtype=tf.int64)

train = tf.convert_to_tensor(train, dtype=tf.int64)


train = tf.data.Dataset.from_tensor_slices(train)

ratings = train.map(lambda x: {
    "item_id": x[0],
    "user_id": x[1],
})

In [17]:
movies = tf.data.Dataset.from_tensor_slices(items)

In [18]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100000, seed=42, reshuffle_each_iteration=False)

split = round(len(shuffled)*.8)

train = shuffled.take(split)
test = shuffled.skip(split).take(len(shuffled)-split)

In [19]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([3967652, 3970819, 3971286, 3973275, 3973379, 3973932, 3976967,
       3977999, 3979755, 3985259])

In [20]:
len(unique_movie_titles)

9130

In [21]:
len(unique_user_ids)

7421

In [28]:
embedding_dimension = 64

In [29]:
user_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [30]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.IntegerLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [31]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["item_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)


model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

model.fit(cached_train, epochs=15)


Epoch 1/15
5/5 [==============================] - 7s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 6.0849e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0012 - factorized_top_k/top_50_categorical_accuracy: 0.0056 - factorized_top_k/top_100_categorical_accuracy: 0.0104 - loss: 58387.6048 - regularization_loss: 0.0000e+00 - total_loss: 58387.6048
Epoch 2/15
5/5 [==============================] - 6s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0051 - factorized_top_k/top_5_categorical_accuracy: 0.0844 - factorized_top_k/top_10_categorical_accuracy: 0.1651 - factorized_top_k/top_50_categorical_accuracy: 0.4443 - factorized_top_k/top_100_categorical_accuracy: 0.5877 - loss: 57984.3978 - regularization_loss: 0.0000e+00 - total_loss: 57984.3978
Epoch 3/15
5/5 [==============================] - 6s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0177 - factorized_top_k/top_5_categorical_accuracy: 0.19

In [32]:
model.evaluate(cached_test)

3/3 [==============================] - 2s 387ms/step - factorized_top_k/top_1_categorical_accuracy: 2.2126e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0028 - factorized_top_k/top_10_categorical_accuracy: 0.0063 - factorized_top_k/top_50_categorical_accuracy: 0.0223 - factorized_top_k/top_100_categorical_accuracy: 0.0394 - loss: 28410.3608 - regularization_loss: 0.0000e+00 - total_loss: 28410.3608


[0.0002212634135503322,
 0.002765792654827237,
 0.006306007504463196,
 0.022347604855895042,
 0.039384886622428894,
 8205.4541015625,
 0,
 8205.4541015625]